# FFF step 1: merge prep

In [ ]:
%load_ext autoreload
%autoreload 2
import hippo
from os import environ, system
from pathlib import Path
import json
from mrich import print
import mrich
import shutil
import molparse as mp

## Get BulkDock HIPPO database

In [ ]:
target_name = "CpKRS"

In [ ]:
animal = hippo.HIPPO(target_name, Path(environ["BULK"])/"TARGETS"/target_name/f"{target_name}.sqlite")

## Define merging hypotheses

In [ ]:
animal.tags.unique

In [ ]:
merging_hypotheses = {
    # nickname: tag,
    "iter1" : "[Other] upload_1 2024-12-12",
}

json.dump(merging_hypotheses, open("merging_hypotheses.json", "wt"))

In [ ]:
# rename things in the README for convenience
system(f"sed -i 's/TARGET_NAME/{target_name}/g' ../README.md")
system(f"sed -i 's/HYPOTHESIS_NICKNAME/{list(merging_hypotheses.keys())[0]}/g' ../README.md")

## Create subdirectories

In [ ]:
for nickname in merging_hypotheses:
    (Path("../fragmenstein") / nickname).mkdir(exist_ok=True, parents=True)
    (Path("../knitwork") / nickname).mkdir(exist_ok=True, parents=True)
    (Path("../syndirella") / nickname).mkdir(exist_ok=True, parents=True)

## Knitwork & Fragmenstein inputs

### Create input SDFs & CSVs

In [ ]:
for nickname, tag in merging_hypotheses.items():
    
    poses = animal.poses(tag=tag)
    mrich.h3(nickname)
    mrich.var("poses", poses)

    # knitwork
    out_dir = Path(f"../knitwork/{nickname}")
    poses.to_knitwork(out_dir / f"{nickname}_input.csv", path_root=out_dir, aligned_files_dir="../aligned_files")

    # # fragmenstein
    poses.write_sdf(out_dir / f"{nickname}_hits.sdf")
    shutil.copy(out_dir / f"{nickname}_hits.sdf", Path(f"../fragmenstein/{nickname}") / f"{nickname}_hits.sdf")

    ref_pose = poses[0]
    mrich.var("ref_pose", ref_pose)
    shutil.copy(ref_pose.apo_path, Path(f"../fragmenstein/{nickname}"))

### Create protonated PDBs for Knitwork

In [ ]:
for nickname, tag in merging_hypotheses.items():
    poses = animal.poses(tag=tag)
    for pose in mrich.track(poses):
        path = pose.apo_path
        sys = mp.parse(path,verbosity=False)
        sys.add_hydrogens(pH=7)
        subdir = Path("../aligned_files")/pose.name
        subdir.mkdir(exist_ok=True, parents=True)
        sys.write(subdir / str(path.name).replace('_apo-desolv.pdb', '_apo-desolv-Hs.pdb'), verbosity=1)

## Backup and close database

In [ ]:
animal.db.backup()
animal.db.close()